# 작업 내용
- time_since_ed, time_to_next, since_start_min -> clipping (max 95%)
- last_trop, run_max_trop의 초기값을 -1에서 0으로 변경
- 최대값(run_max)은 있는데 현재값(last_trop)이 없는 모순적인 상황 해결

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import pandas as pd
import numpy as np
import os

base_path = '/content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스'
cohort_ver124 = pd.read_csv(os.path.join(base_path, 'cohort_ver124_feature_matrix_repaired.csv'))
print(cohort_ver124.columns)
cohort_ver124

Index(['subject_id', 'hadm_id', 'age', 'gender', 'race', 'cci_score',
       'hfrs_score', 'door_to_ecg', 'door_to_trop', 'door_to_anti',
       'door_to_pci', 'prefix_len', 'time_since_ed', 'time_since_last',
       'is_night', 'cum_ecg_cnt', 'cum_stemi_cnt', 'cum_trop_cnt',
       'stemi_flag', 'trop_pos_flag', 'last_trop', 'run_max_trop',
       'trop_trend', 'pci_status', 'pathway_stage', 'time_since_start_min',
       'current_event_id', 'prefix_events_str', 'current_heart_rate',
       'current_mean_bp', 'target_mortality', 'target_next_evt',
       'target_time_to_next', 'target_remain_los', 'arrival_transport'],
      dtype='object')


,subject_id,hadm_id,age,gender,race,cci_score,hfrs_score,door_to_ecg,door_to_trop,door_to_anti,...,time_since_start_min,current_event_id,prefix_events_str,current_heart_rate,current_mean_bp,target_mortality,target_next_evt,target_time_to_next,target_remain_los,arrival_transport
0,13647833,20006266,40,1,ASIAN - SOUTH EAST ASIAN,0,0.0,0,0,0,...,0.0,1,0,0,0,0,4,0.0,458714.683333,1
1,13647833,20006266,40,1,ASIAN - SOUTH EAST ASIAN,0,0.0,0,0,0,...,0.0,4,0,0,0,0,0,0.0,301.033333,1
2,13647833,20006266,40,1,ASIAN - SOUTH EAST ASIAN,0,0.0,0,0,0,...,0.0,0,0,0,0,0,2,626.0,290.133333,1
3,13647833,20006266,40,1,ASIAN - SOUTH EAST ASIAN,0,0.0,0,0,0,...,626.0,2,0,0,0,0,2,614.0,279.700000,1
4,13647833,20006266,40,1,ASIAN - SOUTH EAST ASIAN,0,0.0,0,0,0,...,1240.0,2,0,0,0,0,7,16168.0,269.466667,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14307,16976054,29997425,70,1,WHITE,1,6.6,0,0,0,...,3759.0,2,0,0,0,0,2,589.0,128.816667,1
14308,16976054,29997425,70,1,WHITE,1,6.6,0,0,0,...,4348.0,2,0,0,0,0,2,120.0,119.000000,1
14309,16976054,29997425,70,1,WHITE,1,6.6,0,0,0,...,4468.0,2,0,0,0,0,2,205.0,117.000000,1
14310,16976054,29997425,70,1,WHITE,1,6.6,0,0,0,...,4673.0,2,0,0,0,0,7,6815.0,113.583333,1


In [21]:
import pandas as pd
import numpy as np

def process_requirements_2_5_6(df_input):
    df = df_input.copy()
    print(f"🚀 데이터 수정 시작 (Original Shape: {df.shape})")

    # [필수] 시계열 연산을 위한 정렬 (hadm_id -> 순서)
    sort_cols = ['hadm_id']
    if 'prefix_len' in df.columns:
        sort_cols.append('prefix_len')
    elif 'time_since_start_min' in df.columns:
        sort_cols.append('time_since_start_min')
    df = df.sort_values(by=sort_cols)

    # =========================================================
    # [Requirement 2] Time Variable Clipping (Max 95%)
    # =========================================================
    print("\n⏱️ [Req 2] 시간 변수 Clipping (상위 95%) 및 음수 제거")
    time_cols = ['time_since_ed', 'target_time_to_next', 'time_since_start_min']

    for col in time_cols:
        if col in df.columns:
            # 1. 음수 -> 0 보정 (기본 안전장치)
            df.loc[df[col] < 0, col] = 0

            # 2. 상위 95% 지점 계산
            upper_limit = df[col].quantile(0.95)

            # 3. Clipping 적용
            df[col] = df[col].clip(upper=upper_limit)

            print(f"   - {col}: 상한값 {upper_limit:.1f}로 제한됨 (95%)")

    # =========================================================
    # [Requirement 5 & 6] Troponin 초기값 0 & 논리 수정
    # =========================================================
    print("\n🩸 [Req 5, 6] Troponin 초기값 0 변경 및 논리 재구조화")

    # 수정 대상 컬럼 (ast_trop은 last_trop의 오타로 간주하고 처리)
    trop_cols = ['last_trop', 'run_max_trop']

    # 1. 기존 -1 값을 NaN으로 변경
    # (이전에 -1로 채웠던 것을 다시 비워야 ffill이 정상 작동함)
    for col in trop_cols:
        if col in df.columns:
            df[col] = df[col].replace(-1, np.nan)

    if 'last_trop' in df.columns:
        # 2. [Req 6 해결] Forward Fill (LOCF: Last Observation Carried Forward)
        # "last_trop은 없는데 run_max는 있는" 문제를 해결하기 위해,
        # 검사 기록이 있으면 그 값을 계속 유지시킴
        df['last_trop'] = df.groupby('hadm_id')['last_trop'].ffill()

        # 3. [Req 5 해결] 남은 NaN(진짜 초기값) -> 0으로 변경
        # 입원 초기에 검사 기록이 없는 구간은 0(평균)으로 채움
        df['last_trop'] = df['last_trop'].fillna(0)

        # 4. [Req 6 해결] Run Max 재계산 (Cumulative Max)
        # 꽉 채워진 last_trop을 기반으로 다시 계산하여 논리적 모순 제거
        df['run_max_trop'] = df.groupby('hadm_id')['last_trop'].cummax().fillna(0)

        print("   - Troponin 로직 수정 완료: NaN -> FFill -> Fillna(0) -> Cummax")

        # [검증]
        # last_trop이 0인데 run_max_trop이 0이 아닌 경우가 있는지 확인 (없어야 정상)
        # 단, run_max는 과거의 최대값이므로 last_trop(현재값)보다 클 수는 있음.
        # 중요한 건 "기록이 있는데(run_max>0) last_trop이 끊기면(NaN/0)" 안 되는 것임.
        # 위 로직대로라면 last_trop이 유지되므로 끊기는 현상이 사라짐.

    return df

# =========================================================
# 실행 및 검증
# =========================================================
try:
    if 'cohort_ver124' in globals():
        # 함수 실행
        cohort_ver124 = process_requirements_2_5_6(cohort_ver124)

        # 결과 검증
        print("\n🔎 [최종 검증]")

        # 1. 시간 변수 Max값 확인
        print("1. Time Variable Max Values (Should match 95% limit):")
        print(cohort_ver124[['time_since_ed', 'time_since_start_min']].max())

        # 2. Troponin 초기값 확인
        if 'last_trop' in cohort_ver124.columns:
            print("\n2. Troponin 초기값 분포 (prefix_len=1):")
            # -1은 없어야 하고, 대부분 0이어야 함
            print(cohort_ver124[cohort_ver124['prefix_len'] == 1]['last_trop'].value_counts().head())

        print("\n✅ 작업 완료")
    else:
        print("❌ 'cohort_ver124' 변수가 메모리에 없습니다.")

except Exception as e:
    print(f"❌ 오류 발생: {e}")

🚀 데이터 수정 시작 (Original Shape: (14312, 35))

⏱️ [Req 2] 시간 변수 Clipping (상위 95%) 및 음수 제거
   - time_since_ed: 상한값 11192.7로 제한됨 (95%)
   - target_time_to_next: 상한값 5425.9로 제한됨 (95%)
   - time_since_start_min: 상한값 11192.7로 제한됨 (95%)

🩸 [Req 5, 6] Troponin 초기값 0 변경 및 논리 재구조화
   - Troponin 로직 수정 완료: NaN -> FFill -> Fillna(0) -> Cummax

🔎 [최종 검증]
1. Time Variable Max Values (Should match 95% limit):
time_since_ed           11192.7
time_since_start_min    11192.7
dtype: float64

2. Troponin 초기값 분포 (prefix_len=1):
last_trop
 0.000000    1928
-0.359511       1
Name: count, dtype: int64

✅ 작업 완료


In [24]:
import os

# 1. 저장 경로 설정
base_path = '/content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스'
file_name = 'cohort_ver125_feature_matrix_repaired.csv'
save_path = os.path.join(base_path, file_name)

# 2. CSV 저장 실행
try:
    if 'cohort_ver124' in globals():
        print(f"💾 데이터 저장 시작...")

        # index=False는 불필요한 행 번호(0, 1, 2...)를 저장하지 않도록 합니다.
        cohort_ver124.to_csv(save_path, index=False)

        print(f"🎉 저장 완료!")
        print(f"📂 경로: {save_path}")
        print(f"📊 데이터 크기: {cohort_ver124.shape}")

    else:
        print("❌ 저장할 변수 'cohort_ver124'가 메모리에 없습니다. 이전 단계 코드를 실행해주세요.")

except Exception as e:
    print(f"❌ 저장 중 오류 발생: {e}")

💾 데이터 저장 시작...
🎉 저장 완료!
📂 경로: /content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스/cohort_ver125_feature_matrix_repaired.csv
📊 데이터 크기: (14312, 35)
